In [1]:
import torch

In [3]:
torch.__version__

'0.4.0'

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

In [6]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


In [8]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [9]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data, target
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data,target
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308058

Test set: Average loss: 0.0221, Accuracy: 6382/10000 (63%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.534496

Test set: Average loss: 0.0064, Accuracy: 8786/10000 (87%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.369819

Test set: Average loss: 0.0044, Accuracy: 9177/10000 (91%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.322671

Test set: Average loss: 0.0033, Accuracy: 9375/10000 (93%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.144468

Test set: Average loss: 0.0026, Accuracy: 9508/10000 (95%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.202896

Test set: Average loss: 0.0023, Accuracy: 9557/10000 (95%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.111126

Test set: Average loss: 0.0021, Accuracy: 9616/10000 (96%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.031445

Test set: Average loss: 0.0018, Accuracy: 9642/10000 (96%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.077635

Test set: Average loss: 0.0018, Accuracy: 9670/10000 (96%)

